In [92]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import math
import mne
import pybv

mat = sio.loadmat('Data/AAS010R01.mat')
mat['signal']
mat2 = sio.loadmat('Data/AAS010R02.mat')
mat3 = sio.loadmat('Data/AAS010R03.mat')
mat4 = sio.loadmat('Data/AAS010R04.mat')
mat5 = sio.loadmat('Data/AAS010R05.mat')
mat6 = sio.loadmat('Data/AAS011R01.mat')
mat7 = sio.loadmat('Data/AAS011R02.mat')
mat8 = sio.loadmat('Data/AAS011R03.mat')
mat9 = sio.loadmat('Data/AAS011R04.mat')
mat10 = sio.loadmat('Data/AAS011R05.mat')
mat11 = sio.loadmat('Data/AAS011R06.mat')
mat12 = sio.loadmat('Data/AAS012R01.mat')
mat13 = sio.loadmat('Data/AAS012R02.mat')
mat14 = sio.loadmat('Data/AAS012R03.mat')
mat15 = sio.loadmat('Data/AAS012R04.mat')
mat16 = sio.loadmat('Data/AAS012R05.mat')
mat17 = sio.loadmat('Data/AAS012R06.mat')
mat18 = sio.loadmat('Data/AAS012R07.mat')
mat19 = sio.loadmat('Data/AAS012R08.mat')

In [165]:
mat = mat12
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Tue Dec 03 08:53:37 2002',
 '__version__': '1.0',
 '__globals__': [],
 'filename': array(['AAS012R01.mat'], dtype='<U13'),
 'evalstring': array(['save AAS012R01.mat'], dtype='<U18'),
 'runnr': array([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]], dtype=uint8),
 'trialnr': array([[  0],
        [  0],
        [  0],
        ...,
        [720],
        [720],
        [720]], dtype=uint16),
 'samplenr': array([[    0],
        [    1],
        [    2],
        ...,
        [35097],
        [35098],
        [35099]], dtype=uint16),
 'signal': array([[-32, 544, 608, ..., 496, 400, 304],
        [160, 432, 576, ..., 256, 240, 160],
        [-48,  32, 256, ..., 224, 160,  16],
        ...,
        [240, 544, 608, ..., 112,  96,   0],
        [352, 848, 864, ...,  48,  64, -32],
        [304, 864, 944, ..., 528, 544, 400]], dtype=int16),
 'SourceTime': array([[28827],
        [28827],
      

In [166]:
df = pd.DataFrame(mat['signal']) 
df['Flashing'] =  mat['Flashing']
df['Events'] = (df['Flashing'] - df['Flashing'].shift(1))
df['StimulusType'] = mat['StimulusType']
df['P300'] = (df['StimulusType'] - df['StimulusType'].shift(1)).abs()
df.fillna(0, inplace=True)
df.loc[df['Events']!=0]

KeyError: 'StimulusType'

In [161]:
df.shape

(29568, 68)

In [162]:
eventos = df.loc[df['Events']==1].index
#df_events = pd.DataFrame({'onset': eventos})
tipos = df['P300'].loc[eventos]
dur = 24*np.ones([len(eventos)])
des = []
des += ['Stimulus']*len(eventos)
df_event = pd.DataFrame({'description': tipos})
df_event = df_event.assign(onset=eventos)
df_event = df_event.assign(duration=dur)
df_event['duration'] = df_event['duration'].astype('int64')
df_event['description'] = df_event['description'].astype('int64')
df_event = df_event.assign(type=des)
df_event = df_event[['onset','duration','description','type']]
df_event

,onset,duration,description,type
630,630,24,0,Stimulus
678,678,24,0,Stimulus
726,726,24,0,Stimulus
774,774,24,0,Stimulus
822,822,24,0,Stimulus
...,...,...,...,...
28734,28734,24,0,Stimulus
28782,28782,24,0,Stimulus
28830,28830,24,1,Stimulus
28878,28878,24,0,Stimulus


In [163]:
events = df_event.to_dict(orient='records')
#df.iloc[:, 0:64]
unit = []
unit += ['µV']*64

In [164]:
from pybv import write_brainvision

sfreq = 240

ch_names = ['FC5','FC3','FC1','FCz','FC2','FC4','FC6',
           'C5','C3','C1','Cz','C2','C4','C6',
           'CP5','CP3','CP1','CPz','CP2','CP4','CP6',
           'Fp1','Fpz','Fp2','AF7','AF3','AFz','AF4','AF8',
           'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
           'FT7','FT8','T7','T8','T9','T10','TP7','TP8',
           'P7','P5','P3','P1','Pz','P2','P4','P6','P8',
           'PO7','PO3','POz','PO4','PO8','O1','Oz','O2','Iz']
print(len(ch_names))
#unit = ['µV']

data = df.iloc[:, 0:64].to_numpy()
print(data.shape)

# for further parameters see our API documentation
write_brainvision(data=data.T, sfreq=sfreq, ch_names=ch_names,
                  folder_out="Data/pybv/",
                  fname_base="pybv_file_121",
                  events=events,
                  unit=unit,
                  overwrite=True)

64
(29568, 64)
